In [ ]:
#!pip install duckdb
#!pip install pandas
#!pip install geopandas
#!pip install sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SK

In [17]:
import duckdb
import os
import glob
import pandas as pd
import re
import math
import gc
import time

In [18]:
# === CONFIGURACIÓN ===
CARPETA_DATOS = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral"
ARCHIVO_SALIDA = os.path.join(CARPETA_DATOS, "Delegacion_comb.parquet")

# === BUSCAR ARCHIVOS PARQUET (recursivamente) ===
archivos_parquet = [
    f for f in glob.glob(os.path.join(CARPETA_DATOS, "**", "*.parquet"), recursive=True)
    if not f.endswith(".crc")
]

if not archivos_parquet:
    print("❌ No se encontraron archivos .parquet.")
else:
    print(f"✅ Se encontraron {len(archivos_parquet)} archivos Parquet.")
    
    # === CONECTAR A DUCKDB (sin archivo .db, todo en memoria) ===
    con = duckdb.connect()

    # === LEER Y COMBINAR ===
    # DuckDB entiende patrones tipo *.parquet y concatena automáticamente.
    # Usamos UNION ALL para concatenar todos los archivos.
    query = f"""
        COPY (
            SELECT * FROM read_parquet({archivos_parquet})
        )
        TO '{ARCHIVO_SALIDA}'
        (FORMAT PARQUET, COMPRESSION 'SNAPPY');
    """

    print("\n⏳ Combinando archivos...")
    con.execute(query)
    con.close()

    print(f"\n💾 Parquet combinado guardado en:\n{ARCHIVO_SALIDA}")


✅ Se encontraron 33 archivos Parquet.

⏳ Combinando archivos...

💾 Parquet combinado guardado en:
/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb.parquet


In [19]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [20]:
archivos = [
    f for f in glob.glob(os.path.join(CARPETA_DATOS, "**", "*.parquet"), recursive=True)
    if not f.endswith(".crc") and not f.endswith("Delegacion_comb.parquet")
]

con = duckdb.connect()

# Total de filas en todos los archivos originales
total_originales = con.execute(f"SELECT SUM(count) FROM (SELECT COUNT(*) AS count FROM read_parquet({archivos}) GROUP BY filename)").fetchone()[0]

# Total en el combinado
total_combinado = con.execute(f"SELECT COUNT(*) FROM read_parquet('{ARCHIVO_SALIDA}')").fetchone()[0]



print(f"\n✅ Total original:  {total_originales:,}")
print(f"✅ Total combinado: {total_combinado:,}")
print("🎯 Coinciden" if total_originales == total_combinado else "⚠️ No coinciden, revisa los archivos.")

# Ver las columnas y tipos
print("\n🧱 Esquema:")
print(con.execute(f"DESCRIBE SELECT * FROM read_parquet('{ARCHIVO_SALIDA}')").fetchdf())


con.close()



✅ Total original:  1,469,677
✅ Total combinado: 1,469,677
🎯 Coinciden

🧱 Esquema:
              column_name column_type null   key default extra
0                     fid      DOUBLE  YES  None    None  None
1                   fid_2      DOUBLE  YES  None    None  None
2            calle_numero     VARCHAR  YES  None    None  None
3           codigo_postal     VARCHAR  YES  None    None  None
4                 colonia     VARCHAR  YES  None    None  None
5                alcaldia     VARCHAR  YES  None    None  None
6             sup_terreno      DOUBLE  YES  None    None  None
7        sup_construccion      DOUBLE  YES  None    None  None
8       anio_construccion      DOUBLE  YES  None    None  None
9              instal_esp      BIGINT  YES  None    None  None
10   valor_unitario_suelo      DOUBLE  YES  None    None  None
11            valor_suelo      DOUBLE  YES  None    None  None
12                cve_vus     VARCHAR  YES  None    None  None
13               subsidio      DOUB

Utilizaré "ageb_lon" y "ageb_lat" para hacer el merge con las otras basesde datos.
Antes quitaré los nulos de mi base final

In [21]:
ARCHIVO = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb.parquet"
con = duckdb.connect()

stats = con.execute(f"""
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN ageb_lon IS NULL OR ageb_lat IS NULL THEN 1 ELSE 0 END) AS sin_coordenadas,
  SUM(CASE WHEN ageb_lon IS NOT NULL AND ageb_lat IS NOT NULL THEN 1 ELSE 0 END) AS con_coordenadas
FROM read_parquet('{ARCHIVO}')
""").fetchdf()

print(stats)

con.close()

     total  sin_coordenadas  con_coordenadas
0  1469677             25.0        1469652.0


In [22]:
CARPETA_DATOS = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral"
ARCHIVO_ORIGINAL = os.path.join(CARPETA_DATOS, "Delegacion_comb.parquet")
ARCHIVO_FILTRADO = os.path.join(CARPETA_DATOS, "Delegacion_comb_filtrado.parquet")

con = duckdb.connect()

# Crear una nueva versión filtrada (sin valores nulos en lat/lon)
query = f"""
COPY (
    SELECT *
    FROM read_parquet('{ARCHIVO_ORIGINAL}')
    WHERE ageb_lon IS NOT NULL AND ageb_lat IS NOT NULL
)
TO '{ARCHIVO_FILTRADO}' (FORMAT PARQUET, COMPRESSION 'SNAPPY');
"""

print("⏳ Filtrando datos...")
con.execute(query)

# Verificar resultado
result = con.execute(f"SELECT COUNT(*) FROM read_parquet('{ARCHIVO_FILTRADO}')").fetchone()[0]
print(f"✅ Filtrado completo. Total de filas con coordenadas: {result:,}")

con.close()


⏳ Filtrando datos...
✅ Filtrado completo. Total de filas con coordenadas: 1,469,652


Ahora cambiare el formato de las latitudes y longitudes del Concat de las delegaciones. de INEGI (EPSG:6372) → WGS84 (EPSG:4326)

In [23]:
from pyproj import Transformer
from shapely.geometry import Point

In [24]:
ageb_path = "/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado.parquet"

In [25]:
# ============================================
# CONVERSIÓN DE COORDENADAS INEGI (CCL ITRF2008) → WGS84
# ============================================

# === 1️⃣ CONFIGURACIÓN ===
parquet_path = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado.parquet"
output_path = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado_wgs84.parquet"


In [26]:
# === 2️⃣ LEER PARQUET ===
print(f"📦 Cargando archivo Parquet: {parquet_path}")
df_ageb = pd.read_parquet(parquet_path)
print(f"✅ Archivo cargado: {df_ageb.shape[0]:,} filas, {df_ageb.shape[1]} columnas")

📦 Cargando archivo Parquet: /Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado.parquet
✅ Archivo cargado: 1,469,652 filas, 151 columnas


In [27]:
# === 3️⃣ CREAR TRANSFORMADOR ===
# INEGI usa Lambert Cónica Conforme ITRF2008 = EPSG:6372
transformer = Transformer.from_crs("EPSG:6372", "EPSG:4326", always_xy=True)

In [28]:
# === 4️⃣ VALIDAR COLUMNAS ===
# Ajusta si tus columnas tienen otros nombres
if not {"ageb_lon", "ageb_lat"}.issubset(df_ageb.columns):
    raise ValueError("❌ El archivo debe tener columnas llamadas 'ageb_lon' y 'ageb_lat'.")

# === 5️⃣ FILTRAR FILAS VÁLIDAS ===
mask_valid = df_ageb["ageb_lon"].notna() & df_ageb["ageb_lat"].notna()
print(f"📍 Filas válidas con coordenadas: {mask_valid.sum():,}")

📍 Filas válidas con coordenadas: 1,469,652


In [29]:
# === 6️⃣ TRANSFORMAR COORDENADAS ===
lon_wgs84, lat_wgs84 = transformer.transform(
    df_ageb.loc[mask_valid, "ageb_lon"].astype(float).values,
    df_ageb.loc[mask_valid, "ageb_lat"].astype(float).values
)

In [30]:
# === 7️⃣ CREAR NUEVAS COLUMNAS ===
df_ageb["lon_wgs84"] = pd.NA
df_ageb["lat_wgs84"] = pd.NA
df_ageb.loc[mask_valid, "lon_wgs84"] = lon_wgs84
df_ageb.loc[mask_valid, "lat_wgs84"] = lat_wgs84

# === 8️⃣ VISTA PREVIA ===
print("\n🔍 Ejemplo de coordenadas transformadas:")
print(df_ageb[["ageb_lon", "ageb_lat", "lon_wgs84", "lat_wgs84"]].head(10))


🔍 Ejemplo de coordenadas transformadas:
       ageb_lon       ageb_lat  lon_wgs84  lat_wgs84
0  2.800048e+06  845015.333800 -99.131257  19.577183
1  2.800048e+06  845015.333800 -99.131257  19.577183
2  2.800006e+06  844827.171348 -99.131691  19.575486
3  2.800006e+06  844827.171348 -99.131691  19.575486
4  2.800955e+06  843250.986300 -99.122927  19.561034
5  2.800006e+06  844827.171348 -99.131691  19.575486
6  2.800006e+06  844827.171348 -99.131691  19.575486
7  2.800006e+06  844827.171348 -99.131691  19.575486
8  2.800006e+06  844827.171348 -99.131691  19.575486
9  2.800006e+06  844827.171348 -99.131691  19.575486


In [31]:
# === 9️⃣ GUARDAR PARQUET ACTUALIZADO ===
# Usa compresión Snappy (por defecto) para eficiencia
df_ageb.to_parquet(output_path, index=False)
print(f"\n✅ Archivo guardado con coordenadas WGS84 en: {output_path}")
print("   Columnas agregadas: 'lon_wgs84', 'lat_wgs84'")


✅ Archivo guardado con coordenadas WGS84 en: /Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado_wgs84.parquet
   Columnas agregadas: 'lon_wgs84', 'lat_wgs84'


Voy a renombrar las columnas de los archivos csv de los puntos de interes

In [36]:
# === CONFIGURACIÓN ===
RUTA_BASE = r"/Users/eaha/Documents/TFM/mlops-repo/data/processed/csv"
SOBREESCRIBIR = True 

In [37]:
# Posibles nombres de columnas para detección flexible
NOMBRES_LATITUD = ["lat", "latitude", "latitud", "y"]
NOMBRES_LONGITUD = ["lon", "long", "longitude", "longitud", "x"]

In [38]:
# === FUNCIÓN PRINCIPAL ===
def renombrar_columnas_csv(ruta_csv):
    try:
        df = pd.read_csv(ruta_csv)
    except Exception as e:
        print(f"❌ No se pudo leer {ruta_csv}: {e}")
        return

    columnas_originales = list(df.columns)
    columnas_lower = [c.strip().lower() for c in columnas_originales]

    renombrar = {}

    # Buscar equivalencias en nombres de columnas
    for i, nombre in enumerate(columnas_lower):
        if nombre in [n.lower() for n in NOMBRES_LATITUD]:
            renombrar[columnas_originales[i]] = "latitud"
        elif nombre in [n.lower() for n in NOMBRES_LONGITUD]:
            renombrar[columnas_originales[i]] = "longitud"

    if renombrar:
        print(f"\n📄 Procesando: {os.path.basename(ruta_csv)}")
        print(f"🔁 Renombrando columnas: {renombrar}")
        df.rename(columns=renombrar, inplace=True)

        # Guardar sobrescribiendo
        df.to_csv(ruta_csv, index=False)
        print(f"✅ Archivo sobrescrito: {ruta_csv}")

        # Validar coordenadas si existen ambas columnas
        if "latitud" in df.columns and "longitud" in df.columns:
            try:
                lat_min, lat_max = df["latitud"].min(), df["latitud"].max()
                lon_min, lon_max = df["longitud"].min(), df["longitud"].max()

                print(f"📊 Rango latitud:  {lat_min:.6f} → {lat_max:.6f}")
                print(f"📊 Rango longitud: {lon_min:.6f} → {lon_max:.6f}")

                # Detección básica de formato
                if abs(lat_min) > 90 or abs(lat_max) > 90 or abs(lon_min) > 180 or abs(lon_max) > 180:
                    print("⚠️ Coordenadas fuera de rango típico (posiblemente en metros o en proyección).")
                else:
                    print("✅ Coordenadas parecen estar en grados geográficos (WGS84).")
            except Exception as e:
                print(f"⚠️ No se pudo calcular rango de coordenadas: {e}")
        else:
            print("⚠️ No se encontraron ambas columnas de latitud y longitud después del renombrado.")
    else:
        print(f"⚠️ No se detectaron columnas para renombrar en {os.path.basename(ruta_csv)}.")



In [39]:
# === RECORRER SUBCARPETAS Y APLICAR ===
for root, _, files in os.walk(RUTA_BASE):
    for file in files:
        if file.lower().endswith(".csv"):
            ruta_completa = os.path.join(root, file)
            renombrar_columnas_csv(ruta_completa)


⚠️ No se detectaron columnas para renombrar en cdmx_areas_verdes_2017.csv.

📄 Procesando: areas_verdes_filtrado.csv
🔁 Renombrando columnas: {'latitud': 'latitud', 'longitud': 'longitud'}
✅ Archivo sobrescrito: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/areas_verdes_filtrado.csv
📊 Rango latitud:  19.173251 → 19.562414
📊 Rango longitud: -99.331159 → -98.962573
✅ Coordenadas parecen estar en grados geográficos (WGS84).

📄 Procesando: escuelas_publicas.csv
🔁 Renombrando columnas: {'latitud': 'latitud', 'longitud': 'longitud'}
✅ Archivo sobrescrito: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/escuelas_publicas.csv
📊 Rango latitud:  19.169883 → 19.577081
📊 Rango longitud: -99.334459 → -98.951824
✅ Coordenadas parecen estar en grados geográficos (WGS84).

📄 Procesando: escuelas_privadas_con_coordenadas.csv
🔁 Renombrando columnas: {'lon': 'longitud', 'lat': 'latitud'}
✅ Archivo sobrescrito: /Users/eaha/Documents/TFM/mlops-repo/data/processed/csv/escuelas_privadas/escue

Ahora ya tengo listos todos mis data sets, procederé a realizar el merge de mi archivo 
Delegacion_comb_filtrado_wgs84.parquet como base y agregarle los datos de todos los otros csv que encontramos

In [7]:
import os
from pathlib import Path
import duckdb
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree

In [8]:
# ============== CONFIGURACIÓN ==============
base_parquet = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado_wgs84.parquet")
carpeta_csvs = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/csv")
csvs_interes = [
    "escuelas_privadas/escuelas_privadas_con_coordenadas.csv",
    "hospitales_y_centros_de_salud/hospitales_y_centros_de_salud_con_coordenadas.csv",
    "mb_shp/Metrobus_estaciones_con_coordenadas.csv",
    "stcmetro_shp/STC_Metro_estaciones_utm14n_con_coordenadas.csv",
    "areas_verdes_filtrado.csv",
    "escuelas_publicas.csv"
]
output_dir = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/tmp_chunks_balltree")
output_dir.mkdir(parents=True, exist_ok=True)

output_final = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Merged_Delegacion_final_balltree.parquet")


In [9]:
chunk_size = 50_000           # ajusta si te hace falta menos RAM (p.ej. 20_000)
max_distance_km = 5.0         # radio de match
R_EARTH_M = 6_371_000.0       # radio tierra (m)
max_distance_m = max_distance_km * 1000.0
max_distance_rad = max_distance_m / R_EARTH_M  # para métrica haversine

In [10]:
# ============== CARGAR BASE ==============
# Usamos DuckDB solo para leer la base en chunks (pandas no streamea parquet por filas fácilmente)
con = duckdb.connect()
n_rows = con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_parquet.as_posix()}')").fetchone()[0]
print(f"📦 Archivo base: {n_rows:,} filas")

# Validaciones mínimas
sample = con.execute(f"SELECT * FROM read_parquet('{base_parquet.as_posix()}') LIMIT 1").fetchdf()
if not {"lon_wgs84", "lat_wgs84"}.issubset(sample.columns):
    raise ValueError("❌ El archivo base debe tener columnas 'lon_wgs84' y 'lat_wgs84'.")

📦 Archivo base: 1,469,652 filas


In [11]:
# ============== PREPARAR CSVs (índice BallTree por archivo) ==============
csv_models = []  # lista de dicts: {alias, df, tree, coords_rad, cols_out}
for rel in csvs_interes:
    csv_path = (carpeta_csvs / rel).resolve()
    alias = csv_path.stem.lower().replace(" ", "_")
    if not csv_path.exists():
        print(f"⚠️ CSV no encontrado, se omite: {csv_path}")
        continue

    df = pd.read_csv(csv_path)
    # Validar columnas esperadas
    if not {"latitud", "longitud"}.issubset(df.columns):
        print(f"⚠️ {csv_path.name} no tiene columnas 'latitud' y 'longitud'. Se omite.")
        continue

    # Filtrar filas con coordenadas válidas
    df = df.loc[df["latitud"].notna() & df["longitud"].notna()].copy()
    if df.empty:
        print(f"⚠️ {csv_path.name} sin coordenadas válidas. Se omite.")
        continue

    # Sufijos: renombrar TODAS las columnas del CSV (incluyendo lat/long) para evitar colisiones
    rename_map = {col: f"{col}_{alias}" for col in df.columns}
    df = df.rename(columns=rename_map)

    # Coordenadas en radianes (orden lat, lon para haversine)
    coords_deg = df[[f"latitud_{alias}", f"longitud_{alias}"]].astype(float).values
    coords_rad = np.radians(coords_deg)
    # BallTree con métrica haversine
    tree = BallTree(coords_rad, metric="haversine")

    # Guardar modelo del CSV
    csv_models.append({
        "alias": alias,
        "df": df,                    # dataframe con columnas ya sufijadas
        "tree": tree,
        "coords_rad": coords_rad,
        "cols_out": df.columns.tolist()  # todas las columnas del CSV (sufijadas)
    })
    print(f"🛠️ Índice listo: {alias} ({len(df):,} puntos)")

if not csv_models:
    raise RuntimeError("No hay CSVs válidos para procesar.")

🛠️ Índice listo: escuelas_privadas_con_coordenadas (3,659 puntos)
🛠️ Índice listo: hospitales_y_centros_de_salud_con_coordenadas (27 puntos)
🛠️ Índice listo: metrobus_estaciones_con_coordenadas (324 puntos)
🛠️ Índice listo: stc_metro_estaciones_utm14n_con_coordenadas (195 puntos)
🛠️ Índice listo: areas_verdes_filtrado (2,750 puntos)
🛠️ Índice listo: escuelas_publicas (2,242 puntos)


In [12]:
# ============== PROCESAR EN CHUNKS ==============
chunk_paths = []
for start in range(0, n_rows, chunk_size):
    end = min(start + chunk_size, n_rows)
    print(f"\n🧩 Chunk {start:,} → {end:,} ({end-start:,} filas)")

    base_chunk = con.execute(
        f"SELECT * FROM read_parquet('{base_parquet.as_posix()}') LIMIT {end-start} OFFSET {start}"
    ).fetchdf()

    # Nos aseguramos de no perder el índice original si lo necesitas
    # base_chunk.reset_index(drop=True, inplace=True)

    # Coordenadas del base en radianes (orden lat, lon)
    base_coords_deg = base_chunk[["lat_wgs84", "lon_wgs84"]].astype(float).values
    base_coords_rad = np.radians(base_coords_deg)

    # Para cada CSV, buscamos el vecino más cercano y añadimos columnas
    for model in csv_models:
        alias = model["alias"]
        tree = model["tree"]
        df_csv = model["df"]  # columnas sufijadas, incluido lat/long

        # k=1 vecino más cercano, devuelve distancia (rad) y el índice del CSV
        dist_rad, idx = tree.query(base_coords_rad, k=1)
        dist_rad = dist_rad.reshape(-1)
        idx = idx.reshape(-1)

        # Validar por radio máximo
        within = dist_rad <= max_distance_rad

        # Seleccionar filas del CSV por índice de vecino
        matched = df_csv.iloc[idx].reset_index(drop=True)

        # Convertir distancia a metros
        dist_m = dist_rad * R_EARTH_M

        # Preparar columnas a anexar (todas las del CSV + la distancia)
        # Para filas fuera del radio, ponemos NaN
        for col in df_csv.columns:
            vals = matched[col].copy()
            vals[~within] = np.nan
            base_chunk[col] = vals.values

        base_chunk[f"dist_m_{alias}"] = np.where(within, dist_m, np.nan)

        print(f"   ➕ {alias}: match en {within.sum():,}/{len(within):,} filas (radio ≤ {max_distance_km} km)")

    # Guardar chunk
    out_path = output_dir / f"merged_chunk_{start:08d}_{end:08d}.parquet"
    base_chunk.to_parquet(out_path, index=False)
    chunk_paths.append(out_path)
    print(f"✅ Guardado: {out_path.name} ({len(base_chunk):,} filas)")




🧩 Chunk 0 → 50,000 (50,000 filas)
   ➕ escuelas_privadas_con_coordenadas: match en 50,000/50,000 filas (radio ≤ 5.0 km)
   ➕ hospitales_y_centros_de_salud_con_coordenadas: match en 50,000/50,000 filas (radio ≤ 5.0 km)
   ➕ metrobus_estaciones_con_coordenadas: match en 46,191/50,000 filas (radio ≤ 5.0 km)
   ➕ stc_metro_estaciones_utm14n_con_coordenadas: match en 38,269/50,000 filas (radio ≤ 5.0 km)
   ➕ areas_verdes_filtrado: match en 50,000/50,000 filas (radio ≤ 5.0 km)
   ➕ escuelas_publicas: match en 50,000/50,000 filas (radio ≤ 5.0 km)
✅ Guardado: merged_chunk_00000000_00050000.parquet (50,000 filas)

🧩 Chunk 50,000 → 100,000 (50,000 filas)
   ➕ escuelas_privadas_con_coordenadas: match en 50,000/50,000 filas (radio ≤ 5.0 km)
   ➕ hospitales_y_centros_de_salud_con_coordenadas: match en 50,000/50,000 filas (radio ≤ 5.0 km)
   ➕ metrobus_estaciones_con_coordenadas: match en 46,373/50,000 filas (radio ≤ 5.0 km)
   ➕ stc_metro_estaciones_utm14n_con_coordenadas: match en 41,068/50,000 f

In [13]:
# ============== UNIR CHUNKS Y CHEQUEAR ==============
print("\n📦 Combinando chunks…")
final_df = pd.concat((pd.read_parquet(p) for p in chunk_paths), ignore_index=True)
print(f"📏 Total final: {len(final_df):,} filas (base tenía {n_rows:,})")

final_df.to_parquet(output_final, index=False)
print(f"🎯 Archivo final: {output_final}")



📦 Combinando chunks…
📏 Total final: 1,469,652 filas (base tenía 1,469,652)
🎯 Archivo final: /Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Merged_Delegacion_final_balltree.parquet


In [14]:
import pandas as pd
from pathlib import Path

# Ruta a tu parquet final
output_final = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Merged_Delegacion_final_balltree.parquet")

# Cargar (solo unas columnas para ahorrar memoria)
df = pd.read_parquet(output_final)

# Detectar todas las columnas nuevas (las que vienen de los CSVs)
cols_csv = [c for c in df.columns if any(
    alias in c for alias in ["escuelas_privadas", "hospitales", "metrobus", "stc_metro", "areas_verdes", "escuelas_publicas"]
)]

# Contar cuántas filas tienen TODAS esas columnas vacías
mask_empty = df[cols_csv].isna().all(axis=1)

sin_match = mask_empty.sum()
total = len(df)
print(f"📊 Filas sin ningún match: {sin_match:,} de {total:,} ({sin_match/total:.2%})")


📊 Filas sin ningún match: 457 de 1,469,652 (0.03%)


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path

ModuleNotFoundError: No module named 'geopandas'

In [4]:
# === CONFIGURACIÓN ===
base_parquet = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Delegacion_comb_filtrado_wgs84.parquet")
carpeta_csvs = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/csv")
csvs_interes = [
    "escuelas_privadas/escuelas_privadas_con_coordenadas.csv",
    "hospitales_y_centros_de_salud/hospitales_y_centros_de_salud_con_coordenadas.csv",
    "mb_shp/Metrobus_estaciones_con_coordenadas.csv",
    "stcmetro_shp/STC_Metro_estaciones_utm14n_con_coordenadas.csv",
    "areas_verdes_filtrado.csv",
    "escuelas_publicas.csv"
]

output_dir = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/tmp_chunks")
output_dir.mkdir(parents=True, exist_ok=True)

output_final = Path("/Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Merged_Delegacion_final_1to1.parquet")


In [5]:
# Parámetros
chunk_size = 50_000          # número de filas por chunk
max_distance_km = 5           # radio máximo de búsqueda (en km)
max_distance_m = max_distance_km * 1000

In [6]:
# === 1️⃣ CARGAR ARCHIVO BASE ===
base = pd.read_parquet(base_parquet)
print(f"📦 Archivo base cargado: {len(base):,} filas, {len(base.columns)} columnas")

# Crear geometría manualmente
if not {'lon_wgs84', 'lat_wgs84'}.issubset(base.columns):
    raise ValueError("❌ El archivo base debe tener columnas 'lon_wgs84' y 'lat_wgs84'")

base = gpd.GeoDataFrame(
    base,
    geometry=gpd.points_from_xy(base["lon_wgs84"], base["lat_wgs84"]),
    crs="EPSG:4326"
)
print(f"✅ Geometría creada: {base.crs}, {len(base):,} filas")


📦 Archivo base cargado: 1,469,652 filas, 153 columnas
✅ Geometría creada: EPSG:4326, 1,469,652 filas


In [7]:

# === 2️⃣ FUNCIONES AUXILIARES ===
def preparar_csv(ruta_csv: Path, base_crs):
    """Carga un CSV con columnas 'latitud' y 'longitud' y lo convierte a GeoDataFrame."""
    df = pd.read_csv(ruta_csv)

    # Validar columnas esperadas
    if not {"latitud", "longitud"}.issubset(df.columns):
        raise ValueError(f"⚠️ El archivo {ruta_csv.name} debe tener columnas 'latitud' y 'longitud'.")

    # Crear geometría
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitud"], df["latitud"]),
        crs="EPSG:4326"
    ).to_crs(base_crs)

    return gdf



In [ ]:
"""from pathlib import Path
import geopandas as gpd

# === 3️⃣ PROCESAR EN CHUNKS (versión segura) ===
chunks = range(0, len(base), chunk_size)
chunk_paths = []

for i, offset in enumerate(chunks):
    print(f"\n🧩 Procesando chunk {i+1}/{len(chunks)} ({offset:,} → {offset+chunk_size:,})")
    base_chunk = base.iloc[offset:offset + chunk_size].copy()

    # Crear geometría si no existe
    if 'geometry' not in base_chunk.columns:
        base_chunk = gpd.GeoDataFrame(
            base_chunk,
            geometry=gpd.points_from_xy(base_chunk["lon_wgs84"], base_chunk["lat_wgs84"]),
            crs="EPSG:4326"
        )

    # Convertir a CRS métrico (para usar distancias en metros)
    base_chunk = base_chunk.to_crs(3857)

    # === MERGE CON CADA CSV ===
    for csv_rel in csvs_interes:
        ruta_csv = carpeta_csvs / csv_rel
        alias = ruta_csv.stem.replace(" ", "_")

        print(f"📍 Merge espacial con: {alias}")
        csv_gdf = preparar_csv(ruta_csv, base_chunk.crs)

        # Renombrar columnas de coordenadas antes del join
        csv_gdf = csv_gdf.rename(columns={
            "latitud": f"latitud_{alias}",
            "longitud": f"longitud_{alias}"
        })

        # merge left (preserva todas las filas del base_chunk)
        merged = gpd.sjoin_nearest(
            base_chunk,
            csv_gdf,
            how="left",
            max_distance=max_distance_m,
            distance_col=f"dist_{alias}",
            rsuffix=f"_{alias}"
        )

        # Renombrar columnas nuevas con sufijo
        new_cols = [c for c in csv_gdf.columns if c not in ['geometry', 'latitud', 'longitud']]
        rename_map = {c: f"{c}_{alias}" for c in new_cols if c in merged.columns}
        merged.rename(columns=rename_map, inplace=True)

        # Eliminar columnas duplicadas de geometría
        merged = merged.loc[:, ~merged.columns.duplicated(keep='first')]

        # 👉 En lugar de reemplazar el chunk completo, actualizamos columnas
        base_chunk = merged

    # Convertir de nuevo a WGS84 para guardar
    base_chunk = base_chunk.to_crs(4326)

    # === GUARDAR RESULTADO PARCIAL ===
    output_chunk = output_dir / f"merged_chunk_{i+1}.parquet"
    base_chunk.to_parquet(output_chunk, index=False)
    chunk_paths.append(output_chunk)
    print(f"✅ Chunk guardado: {output_chunk.name} ({len(base_chunk):,} filas)")"""



🧩 Procesando chunk 1/30 (0 → 50,000)
📍 Merge espacial con: escuelas_privadas_con_coordenadas
📍 Merge espacial con: hospitales_y_centros_de_salud_con_coordenadas
📍 Merge espacial con: Metrobus_estaciones_con_coordenadas
📍 Merge espacial con: STC_Metro_estaciones_utm14n_con_coordenadas


/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_1902/1339388365.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitud_left', 'latitud_left'} in the result is deprecated and will raise a MergeError in a future version.
  merged = gpd.sjoin_nearest(


📍 Merge espacial con: areas_verdes_filtrado
📍 Merge espacial con: escuelas_publicas


/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_1902/1339388365.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitud_left', 'latitud_left'} in the result is deprecated and will raise a MergeError in a future version.
  merged = gpd.sjoin_nearest(


✅ Chunk guardado: merged_chunk_1.parquet (66,029 filas)

🧩 Procesando chunk 2/30 (50,000 → 100,000)
📍 Merge espacial con: escuelas_privadas_con_coordenadas
📍 Merge espacial con: hospitales_y_centros_de_salud_con_coordenadas
📍 Merge espacial con: Metrobus_estaciones_con_coordenadas
📍 Merge espacial con: STC_Metro_estaciones_utm14n_con_coordenadas


...

✅ Chunk guardado: merged_chunk_29.parquet (66,672 filas)

🧩 Procesando chunk 30/30 (1,450,000 → 1,500,000)
📍 Merge espacial con: escuelas_privadas_con_coordenadas
📍 Merge espacial con: hospitales_y_centros_de_salud_con_coordenadas
📍 Merge espacial con: Metrobus_estaciones_con_coordenadas
📍 Merge espacial con: STC_Metro_estaciones_utm14n_con_coordenadas
📍 Merge espacial con: areas_verdes_filtrado


/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_1902/1339388365.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitud_left', 'latitud_left'} in the result is deprecated and will raise a MergeError in a future version.
  merged = gpd.sjoin_nearest(


📍 Merge espacial con: escuelas_publicas


/var/folders/5x/v8hk2rj9117bkb38sq4yjyfw0000gn/T/ipykernel_1902/1339388365.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitud_left', 'latitud_left'} in the result is deprecated and will raise a MergeError in a future version.
  merged = gpd.sjoin_nearest(


✅ Chunk guardado: merged_chunk_30.parquet (26,341 filas)


In [8]:
# === 3️⃣ PROCESAR EN CHUNKS (versión sin cambio de CRS, 1:1 real) ===

chunks = range(0, len(base), chunk_size)
chunk_paths = []

# Convertir distancia de km a grados (aprox.)
max_distance_deg = max_distance_km / 111

for i, offset in enumerate(chunks):
    print(f"\n🧩 Procesando chunk {i+1}/{len(chunks)} ({offset:,} → {offset+chunk_size:,})")
    base_chunk = base.iloc[offset:offset + chunk_size].copy()

    # Asegurar que tiene geometría válida
    if "geometry" not in base_chunk.columns:
        base_chunk = gpd.GeoDataFrame(
            base_chunk,
            geometry=gpd.points_from_xy(base_chunk["lon_wgs84"], base_chunk["lat_wgs84"]),
            crs="EPSG:4326"
        )

    # === MERGE CON CADA CAPA CSV ===
    for csv_rel in csvs_interes:
        ruta_csv = carpeta_csvs / csv_rel
        alias = ruta_csv.stem.replace(" ", "_")

        print(f"📍 Merge espacial con: {alias}")
        csv_gdf = preparar_csv(ruta_csv, base_chunk.crs)

        # Eliminar columna conflictiva si quedó de un merge previo
        if "index_right" in base_chunk.columns:
            base_chunk = base_chunk.drop(columns=["index_right"])

        # === JOIN ESPACIAL 1:1 POR PUNTO MÁS CERCANO ===
        merged = gpd.sjoin_nearest(
            base_chunk,
            csv_gdf,
            how="left",
            max_distance=max_distance_deg,  # usar grados, no metros
            distance_col=f"dist_{alias}",
            rsuffix=f"_{alias}"
        )

        # --- LIMPIEZA Y RENOMBRADO DE COLUMNAS ---
        merged = pd.DataFrame(merged)

        # Renombrar columnas nuevas con sufijo (excepto coordenadas y geometry)
        new_cols = [c for c in csv_gdf.columns if c not in ["geometry", "latitud", "longitud"]]
        rename_map = {c: f"{c}_{alias}" for c in new_cols if c in merged.columns}
        merged.rename(columns=rename_map, inplace=True)

        merged = merged.loc[:, ~merged.columns.duplicated()]

        # ✅ Conservar todas las filas del base_chunk
        base_chunk = pd.merge(
            base_chunk,
            merged[
                ["lon_wgs84", "lat_wgs84"]
                + [col for col in merged.columns if col.startswith(tuple(rename_map.values())) or col.startswith("dist_")]
            ],
            on=["lon_wgs84", "lat_wgs84"],
            how="left"
        )

        # Volver a GeoDataFrame
        base_chunk = gpd.GeoDataFrame(base_chunk, geometry=gpd.points_from_xy(base_chunk["lon_wgs84"], base_chunk["lat_wgs84"]), crs="EPSG:4326")

    # === GUARDAR RESULTADO PARCIAL ===
    output_chunk = output_dir / f"merged_chunk_{i+1}.parquet"
    base_chunk.to_parquet(output_chunk, index=False)
    chunk_paths.append(output_chunk)
    print(f"✅ Chunk guardado: {output_chunk.name} ({len(base_chunk):,} filas)")

# === RESUMEN FINAL ===
print("\n📦 Procesamiento terminado.")
print(f"🧩 Se generaron {len(chunk_paths)} archivos parquet en: {output_dir}")



🧩 Procesando chunk 1/30 (0 → 50,000)
📍 Merge espacial con: escuelas_privadas_con_coordenadas


/Users/eaha/Documents/TFM/.venv/lib/python3.12/site-packages/geopandas/array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


: 

In [39]:
# === 4️⃣ UNIR TODOS LOS CHUNKS ===
print("\n🧩 Combinando todos los chunks finales...")
df_final = pd.concat([pd.read_parquet(p) for p in chunk_paths], ignore_index=True)
print(f"✅ Total final: {len(df_final):,} filas")

df_final.to_parquet(output_final, index=False)
print(f"🎯 Archivo final guardado en: {output_final}")


🧩 Combinando todos los chunks finales...
✅ Total final: 36,573 filas
🎯 Archivo final guardado en: /Users/eaha/Documents/TFM/mlops-repo/data/processed/AGEBvsCatastral/Merged_Delegacion_final_1to1.parquet
